# Multi-Layer Perceptron
## MLP 함수 구현

<img width="985" alt="mlp-func" src="https://user-images.githubusercontent.com/28593767/113245908-e2381800-92f2-11eb-8a03-f608cbfd66ee.png">

In [1]:
import numpy as np

hidden_cnt = 3
input_cnt = 7
output_cnt = 1

RND_MEAN = 0
RND_STD = 0.003

def init_model_hidden1():
    global pm_output, pm_hidden, input_cnt, output_cnt, hidden_cnt
    
    pm_hidden = allocate_param_pair([input_cnt,hidden_cnt]) 
    pm_output = allocate_param_pair([hidden_cnt,output_cnt])
    
def init_model_hiddens():
    global pm_output, pm_hiddens, input_cnt, output_cnt, hidden_config
    
    pm_hiddens = []
    prev_cnt = input_cnt
    
    for hidden_cnt in hidden_config:
        pm_hiddens.append(alloc_param_pair([prev_cnt, hidden_cnt]))
        prev_cnt = hidden_cnt
    
    pm_output = alloc_param_pair([prev_cnt, output_cnt])    

In [2]:
def allocate_param_pair(shape):
    weight = np.random.normal(RND_MEAN, RND_STD, shape)
    bias = np.zeros(shape[-1])
    
    return {'w' : weight, 'b' : bias}

In [3]:
def forward_neuralnet_hidden1(x): 
    global pm_output, pm_hidden
    hidden = relu(np.matmul(x,pm_hidden['w']) + pm_hidden['b']) 
    output = np.matmul(hidden,pm_output['w']) + pm_output['b']
    
    return output, [x, hidden]

def relu(x):
    return np.maximum(x,0)

In [4]:
def backprop_neuralnet_hidden1(G_output, aux):
    global pm_hidden, pm_output
    x, hidden = aux
    
    g_output_w_out = hidden.transpose()
    G_w_output = np.matmul(g_output_w_out, G_output)
    G_b_output = np.sum(G_output, axis = 0)
    
    g_output_hidden = pm_output['w'].transpose()
    G_hidden = np.matmul(G_output, g_output_hidden)
    
    # Update output layer parameter
    pm_output['w'] -= LEARNING_RATE * G_w_output
    pm_output['b'] -= LEARNING_RATE * G_b_output
    
    G_hidden = G_hidden * relu_derv(hidden)
    
    g_hidden_w_hid = x.transpose()

    # Update hidden layer parameter
    G_w_hid = np.matmul(g_hidden_w_hid, G_hidden) 
    G_b_hid = np.sum(G_hidden, axis = 0)
    
    pm_hidden['w'] -= LEARNING_RATE * G_w_hid 
    pm_hidden['b'] -= LEARNING_RATE * G_b_hid

In [5]:
def relu_derv(y):
    return np.sign(y)

In [6]:
def forward_neuralnet_hiddens(x):
    global pm_output, pm_hiddens
    
    hidden = x
    hiddens = [x]
    
    for pm_hidden in pm_hiddens:
        hidden = relu(np.matmul(hidden, pm_hidden['w']) + pm_hidden['b'])
        hiddens.append(hidden)
        
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, hiddens

In [7]:
def backprop_neuralnet_hiddens(G_output, aux):
    global pm_output, pm_hiddens

    hiddens = aux
    
    g_output_w_out = hiddens[-1].transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis=0)

    g_output_hidden = pm_output['w'].transpose() 
    G_hidden = np.matmul(G_output, g_output_hidden)

    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out
    
    for n in reversed(range(len(pm_hiddens))):
        G_hidden = G_hidden * relu_derv(hiddens[n+1])

        g_hidden_w_hid = hiddens[n].transpose()
        G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
        G_b_hid = np.sum(G_hidden, axis=0)
    
        g_hidden_hidden = pm_hiddens[n]['w'].transpose()
        G_hidden = np.matmul(G_hidden, g_hidden_hidden)

        pm_hiddens[n]['w'] -= LEARNING_RATE * G_w_hid
        pm_hiddens[n]['b'] -= LEARNING_RATE * G_b_hid

In [8]:
def set_hidden(info):
    global hidden_cnt, hidden_config 
    if isinstance(info, int):
        hidden_cnt = info
        hidden_config = None 
    else:
        hidden_config = info